In [1]:
import numpy as np
from functions import prepData, splitData
from crossValidation import crossValidate
from Training import TrainDTree, SplitInfo, selectBestFeature
from PredictPrune import PredictDTree, PruneDTree

In [2]:
raw = np.array([('Sunny', 'Hot', 'High', 'False', 'N'),
                ('Sunny', 'Hot', 'High', 'True', 'N'),
                ('Overcast', 'Hot', 'High', 'False', 'P'), 
                ('Rainy', 'Mild', 'High', 'False', 'P'), 
                ('Rainy', 'Cool', 'Normal', 'False', 'P'), 
                ('Rainy', 'Cool', 'Normal', 'True', 'N'), 
                ('Overcast', 'Cool', 'Normal', 'True', 'P'), 
                ('Sunny', 'Mild', 'High', 'False', 'N'), 
                ('Sunny', 'Cool', 'Normal', 'False', 'P'), 
                ('Rainy', 'Mild', 'Normal', 'False', 'P'), 
                ('Sunny', 'Mild', 'Normal', 'True', 'P'),
                ('Overcast', 'Mild', 'High', 'True', 'P'),
                ('Overcast', 'Hot', 'Normal', 'False', 'P'), 
                ('Rainy', 'Mild', 'High', 'True', 'N')],
               dtype=[('Outlook','U8'), ('Temperature','U4'), ('Humidity','U6'),
                      ('Windy','U5'), ('Class','U1')]
            )
testData = raw[['Outlook','Temperature','Humidity','Windy']]
testLabel = raw['Class']

In [3]:
SplitInfo(testData['Temperature'], testLabel)

(0.9110633930116763, 1.5566567074628228, None)

In [68]:
selectBestFeature(testData, testLabel, useRatio=False)

('Outlook', 0.24674981977443933, None, 0.24674981977443933)

In [69]:
testTree = TrainDTree(testData,testLabel)
print(testTree)

Attribute [Outlook]:
    value = Overcast, class: P
    value = Sunny, Attribute [Humidity]:
        value = Normal, class: P
        value = High, class: N
    value = Rainy, Attribute [Windy]:
        value = False, class: P
        value = True, class: N



In [3]:
carPath = './data/car.data'
carNames = ['buying','maint','doors','persons','lug_boot','safety','accept']
carData, carFeats, acceptVec = prepData(carPath, carNames, slice(-1), 'accept')

(xvData,xvLabel),xvFolds,pruningSet = splitData(carData, acceptVec)
testData,testLabel = xvData[xvFolds[0]],xvLabel[xvFolds[0]]
trainIdx = np.hstack(xvFolds[1:])
trainData,trainLabel = xvData[trainIdx],xvLabel[trainIdx]

(array([0.09615385, 0.13141026, 0.09967846, 0.11935484, 0.14239482]),
 array([0.08653846, 0.12179487, 0.10289389, 0.11935484, 0.10355987]))

In [ ]:
z = TrainDTree(trainData, trainLabel)
z.combineChildNodes()
print(z)

In [4]:
segPD = './data/segmentation.data'
segVars = ['class', 'centCol', 'centRow', 'pixelCount', 'slDensity5', 
            'slDensity2', 'vedgeMean', 'vegdeSD', 'hedgeMean', 'hedgeSD',
            'intenseMean', 'rawredMean', 'rawblueMean', 'rawgreenMean',
             'exredMean', 'exblueMean', 'exgreenMean', 'valueMean', 
             'saturateMean', 'hueMean']
segFeatIdx = [1,2] + list(range(4,19))
segData, segFeats, segClass = prepData(segPD, segVars, segFeatIdx, 'class')

(xvData,xvLabel),xvFolds,pruningSet = splitData(segData, segClass)
crossValidate(xvData, xvLabel, pruningSet, xvFolds)

(array([0.16666667, 0.23809524, 0.17142857, 0.25714286, 0.17142857]),
 array([0.16666667, 0.21428571, 0.17142857, 0.25714286, 0.17142857]))

In [54]:
abalonePath = './data/abalone.data'
abaloneNames = ['sex', 'length', 'diameter', 'height', 'wholeHt',
                'shuckWt', 'visceraWt', 'shellWt', 'rings']
abaloneData, abaloneFeats, ringVec = prepData(abalonePath,
                                              abaloneNames, slice(-1),'rings')
ringLabel = ringVec.astype(str)
ringLabel[ringVec<=5] = '<5'
ringLabel[ringVec>=16] = '16+'
(xvData,xvLabel),xvFolds,pruningSet = splitData(abaloneData, ringLabel)

In [55]:
testData,testLabel = xvData[xvFolds[0]],xvLabel[xvFolds[0]]
trainIdx = np.hstack(xvFolds[1:])
trainData,trainLabel = xvData[trainIdx],xvLabel[trainIdx]

z = TrainDTree(trainData, trainLabel)
z.combineChildNodes()
print(z)

Attribute [shellWt]:
    value < 0.159250, Attribute [diameter]:
        value < 0.222500, Attribute [visceraWt]:
            value < 0.011500, Attribute [length]:
                value < 0.252500, Attribute [height]:
                    value < 0.055000, Attribute [wholeHt]:
                        value < 0.041250, class: <5
                        value >= 0.041250, Attribute [sex]:
                            value = M, class: 7
                            value = I, class: <5
                    value >= 0.055000, Attribute [wholeHt]:
                        value < 0.026750, class: 6
                        value >= 0.026750, class: <5
                value >= 0.252500, class: 7
            value >= 0.011500, Attribute [shuckWt]:
                value < 0.046000, Attribute [length]:
                    value < 0.287500, class: <5
                    value >= 0.287500, Attribute [height]:
                        value < 0.067500, Attribute [wholeHt]:
                            va

389

In [ ]:
PruneDTree(z, pruningSet[0], pruningSet[1])
print(z)